In [90]:
%load_ext autoreload
%autoreload 2

import __confing
__confing.change_to_root_folder()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current directory: c:\Users\Leo\OneDrive\Escritorio\Final Proyect


In [91]:
try:
    __confing.execute_notebook('notebooks/data_extract.ipynb')
except:
    pass

In [92]:
import pandas as pd
import numpy as np

from src import value_filler as vf
from src import feature_selection as fs
from src import data_procces as dp

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from src import evaluation

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error,roc_auc_score,confusion_matrix
from sklearn.model_selection import KFold

In [93]:
def feature_importance(model, X: pd.DataFrame, y: pd.Series, n: int):
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    
    feature_names_original = list(X.columns)
    
    target_name = y.name  # Obtener el nombre de la columna objetivo desde la Serie y
    
    if target_name in feature_names_original:
        feature_names_original.remove(target_name)
    
    max_importance = list(indices[:n])
    
    nueva_lista = [feature_names_original[i] for i in max_importance]
    
    return nueva_lista


In [94]:
n='a'
df=pd.read_csv(r'data\custom\all_waves.csv')
target=f'p{n}hosp1y'

In [95]:
selected = ['pahosp1y','padoctor1y','paurina2y','pacholst','pabreast','paprost','papapsm','pamammog','pacage','padrinkcr','padrinkbd','padrinkb','paheight','paweight','pabmi','papaina','papainlv','papainfr','pafatigue','pawheeze','pabreath_m','paswell','pahearaid','pahearing','pasight','paglasses','palunglmt_m','pahrtatlmt','pastroklmt','paarthlmt','palowermoba','pauppermoba','pagrossaa','palgmusaa','pamobilaa','paclims','pasit','paarms','palift','pastoop','pachair','pameals','pashop','pamoney','pameds','pabedhlp','pabed','patoilt','paeat','pabath','pawalkr','padress','pahipcomp','pafall','pafallinj','pamhip','pahipe_m','parxarthr','parxstrok','parxhrtat','pacncrothr','pacncrmeds','pacncrradn','pacncrsurg','pacncrchem','parxdiab','parxdiabo','parxhibp','parechrtatt','parifaany','parfaany','parfcaren','pararcare','padresshlp', 'pawalkhlp', 'pabathehlp', 'paeathlp', 'pabedhlp', 'patoilethlp', 'pamealhlp', 'pashophlp', 'pamedhlp', 'pamoneyhlp','pahibpe','pagender','padiabe','pacancre','palunglmt_m','parxlung_m','pahrtatte','pastroke','paarthre','parifaany','parafaany','pahigov','pasmokev','pasmoken','paoangry','paosleep','paodngr','paodngr','paopace','paoplot','paoalchl','pawthh','paagey','pamomage','padadage','paprmem','parjudg','parorgnz']
len(selected)

111

In [96]:
selected=f85

In [97]:
selected = list(set(selected))

df = df.dropna(subset=[target]) 
df = df[selected].copy()

In [98]:
porcentaje_nulos = df.isnull().sum(axis=1) / len(df.columns)
df = df[(df[target] != 0) | (porcentaje_nulos <= 0.5)]

In [99]:
# fill
y = df[target]
X = df.drop(target,axis=1)

In [100]:
from src import feature_selection as fs
X = fs.fast_fill(X)

In [101]:
# split
X_train, X_test, y_train, y_test=dp.split_data(X, y, test_size=0.2, random_state=40)

In [102]:
# smote
X_train, y_train = dp.apply_smote(X_train, y_train)

In [103]:
# scaler
X_train, X_test = dp.apply_standard_scaler(X_train, X_test)

In [104]:
params={'max_depth': 28, 'subsample': 0.6559545947005226, 'colsample_bytree': 0.9075616505206274, 'eta': 0.03568753700048324, 'min_child_weight': 2, 'learning_rate': 0.0869908981133205, 'n_estimators': 571}

In [105]:
model = XGBClassifier()
model.fit(X_train, y_train)

y_pred_proba = model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)
print(f'AUC={auc}')
print(y_test.value_counts())

AUC=0.7973208453205527
pahosp1y
0.0    20763
1.0     2862
Name: count, dtype: int64


In [ ]:
raise ValueError('STOP')

# FEATURE SELECTION

In [35]:
f50=feature_importance(model, X, y, 50)
f50.append('pahosp1y')
print(f50)

['pagrossaa', 'palift', 'padoctor1y', 'pacholst', 'parxhibp', 'pafatigue', 'parafaany', 'parfcaren', 'pabreath_m', 'pamobilaa', 'paoalchl', 'paswell', 'pagender', 'papainfr', 'parifaany', 'pasit', 'pasmokev', 'pafall', 'pashop', 'pahrtatte', 'paprost', 'pararcare', 'pahipe_m', 'parfaany', 'parxdiab', 'pafallinj', 'pabathehlp', 'palgmusaa', 'pahigov', 'pamammog', 'pasmoken', 'paodngr', 'pacancre', 'padrinkcr', 'paeat', 'papaina', 'papapsm', 'pacncrsurg', 'paarthre', 'parxdiabo', 'patoilt', 'parxhrtat', 'pahearing', 'pacncrchem', 'paarthlmt', 'pameals', 'paglasses', 'parxlung_m', 'padiabe', 'pasight', 'pahosp1y']


In [36]:
f55=feature_importance(model, X, y, 55)
f55.append('pahosp1y')
print(f55)

['pagrossaa', 'palift', 'padoctor1y', 'pacholst', 'parxhibp', 'pafatigue', 'parafaany', 'parfcaren', 'pabreath_m', 'pamobilaa', 'paoalchl', 'paswell', 'pagender', 'papainfr', 'parifaany', 'pasit', 'pasmokev', 'pafall', 'pashop', 'pahrtatte', 'paprost', 'pararcare', 'pahipe_m', 'parfaany', 'parxdiab', 'pafallinj', 'pabathehlp', 'palgmusaa', 'pahigov', 'pamammog', 'pasmoken', 'paodngr', 'pacancre', 'padrinkcr', 'paeat', 'papaina', 'papapsm', 'pacncrsurg', 'paarthre', 'parxdiabo', 'patoilt', 'parxhrtat', 'pahearing', 'pacncrchem', 'paarthlmt', 'pameals', 'paglasses', 'parxlung_m', 'padiabe', 'pasight', 'papainlv', 'parjudg', 'padrinkbd', 'pacncrothr', 'palunglmt_m', 'pahosp1y']


In [37]:
f65=feature_importance(model, X, y, 65)
f65.append('pahosp1y')
print(f65)

['pagrossaa', 'palift', 'padoctor1y', 'pacholst', 'parxhibp', 'pafatigue', 'parafaany', 'parfcaren', 'pabreath_m', 'pamobilaa', 'paoalchl', 'paswell', 'pagender', 'papainfr', 'parifaany', 'pasit', 'pasmokev', 'pafall', 'pashop', 'pahrtatte', 'paprost', 'pararcare', 'pahipe_m', 'parfaany', 'parxdiab', 'pafallinj', 'pabathehlp', 'palgmusaa', 'pahigov', 'pamammog', 'pasmoken', 'paodngr', 'pacancre', 'padrinkcr', 'paeat', 'papaina', 'papapsm', 'pacncrsurg', 'paarthre', 'parxdiabo', 'patoilt', 'parxhrtat', 'pahearing', 'pacncrchem', 'paarthlmt', 'pameals', 'paglasses', 'parxlung_m', 'padiabe', 'pasight', 'papainlv', 'parjudg', 'padrinkbd', 'pacncrothr', 'palunglmt_m', 'parorgnz', 'pabreast', 'pastroke', 'pahrtatlmt', 'parxstrok', 'pahearaid', 'pashophlp', 'parxarthr', 'pawalkhlp', 'pabedhlp', 'pahosp1y']


In [38]:
f70=feature_importance(model, X, y, 70)
f70.append('pahosp1y')
print(f70)

['pagrossaa', 'palift', 'padoctor1y', 'pacholst', 'parxhibp', 'pafatigue', 'parafaany', 'parfcaren', 'pabreath_m', 'pamobilaa', 'paoalchl', 'paswell', 'pagender', 'papainfr', 'parifaany', 'pasit', 'pasmokev', 'pafall', 'pashop', 'pahrtatte', 'paprost', 'pararcare', 'pahipe_m', 'parfaany', 'parxdiab', 'pafallinj', 'pabathehlp', 'palgmusaa', 'pahigov', 'pamammog', 'pasmoken', 'paodngr', 'pacancre', 'padrinkcr', 'paeat', 'papaina', 'papapsm', 'pacncrsurg', 'paarthre', 'parxdiabo', 'patoilt', 'parxhrtat', 'pahearing', 'pacncrchem', 'paarthlmt', 'pameals', 'paglasses', 'parxlung_m', 'padiabe', 'pasight', 'papainlv', 'parjudg', 'padrinkbd', 'pacncrothr', 'palunglmt_m', 'parorgnz', 'pabreast', 'pastroke', 'pahrtatlmt', 'parxstrok', 'pahearaid', 'pashophlp', 'parxarthr', 'pawalkhlp', 'pabedhlp', 'paclims', 'paurina2y', 'pawheeze', 'pachair', 'padrinkb', 'pahosp1y']


In [39]:
f75=feature_importance(model, X, y, 75)
f75.append('pahosp1y')
print(f75)

['pagrossaa', 'palift', 'padoctor1y', 'pacholst', 'parxhibp', 'pafatigue', 'parafaany', 'parfcaren', 'pabreath_m', 'pamobilaa', 'paoalchl', 'paswell', 'pagender', 'papainfr', 'parifaany', 'pasit', 'pasmokev', 'pafall', 'pashop', 'pahrtatte', 'paprost', 'pararcare', 'pahipe_m', 'parfaany', 'parxdiab', 'pafallinj', 'pabathehlp', 'palgmusaa', 'pahigov', 'pamammog', 'pasmoken', 'paodngr', 'pacancre', 'padrinkcr', 'paeat', 'papaina', 'papapsm', 'pacncrsurg', 'paarthre', 'parxdiabo', 'patoilt', 'parxhrtat', 'pahearing', 'pacncrchem', 'paarthlmt', 'pameals', 'paglasses', 'parxlung_m', 'padiabe', 'pasight', 'papainlv', 'parjudg', 'padrinkbd', 'pacncrothr', 'palunglmt_m', 'parorgnz', 'pabreast', 'pastroke', 'pahrtatlmt', 'parxstrok', 'pahearaid', 'pashophlp', 'parxarthr', 'pawalkhlp', 'pabedhlp', 'paclims', 'paurina2y', 'pawheeze', 'pachair', 'padrinkb', 'paeathlp', 'paarms', 'pawthh', 'pastroklmt', 'pawalkr', 'pahosp1y']


In [40]:
f80=feature_importance(model, X, y, 80)
f80.append('pahosp1y')
print(f80)

['pagrossaa', 'palift', 'padoctor1y', 'pacholst', 'parxhibp', 'pafatigue', 'parafaany', 'parfcaren', 'pabreath_m', 'pamobilaa', 'paoalchl', 'paswell', 'pagender', 'papainfr', 'parifaany', 'pasit', 'pasmokev', 'pafall', 'pashop', 'pahrtatte', 'paprost', 'pararcare', 'pahipe_m', 'parfaany', 'parxdiab', 'pafallinj', 'pabathehlp', 'palgmusaa', 'pahigov', 'pamammog', 'pasmoken', 'paodngr', 'pacancre', 'padrinkcr', 'paeat', 'papaina', 'papapsm', 'pacncrsurg', 'paarthre', 'parxdiabo', 'patoilt', 'parxhrtat', 'pahearing', 'pacncrchem', 'paarthlmt', 'pameals', 'paglasses', 'parxlung_m', 'padiabe', 'pasight', 'papainlv', 'parjudg', 'padrinkbd', 'pacncrothr', 'palunglmt_m', 'parorgnz', 'pabreast', 'pastroke', 'pahrtatlmt', 'parxstrok', 'pahearaid', 'pashophlp', 'parxarthr', 'pawalkhlp', 'pabedhlp', 'paclims', 'paurina2y', 'pawheeze', 'pachair', 'padrinkb', 'paeathlp', 'paarms', 'pawthh', 'pastroklmt', 'pawalkr', 'pauppermoba', 'paagey', 'pabmi', 'pastoop', 'palowermoba', 'pahosp1y']


In [41]:
f85=feature_importance(model, X, y, 85)
f85.append('pahosp1y')
print(f85)

['pagrossaa', 'palift', 'padoctor1y', 'pacholst', 'parxhibp', 'pafatigue', 'parafaany', 'parfcaren', 'pabreath_m', 'pamobilaa', 'paoalchl', 'paswell', 'pagender', 'papainfr', 'parifaany', 'pasit', 'pasmokev', 'pafall', 'pashop', 'pahrtatte', 'paprost', 'pararcare', 'pahipe_m', 'parfaany', 'parxdiab', 'pafallinj', 'pabathehlp', 'palgmusaa', 'pahigov', 'pamammog', 'pasmoken', 'paodngr', 'pacancre', 'padrinkcr', 'paeat', 'papaina', 'papapsm', 'pacncrsurg', 'paarthre', 'parxdiabo', 'patoilt', 'parxhrtat', 'pahearing', 'pacncrchem', 'paarthlmt', 'pameals', 'paglasses', 'parxlung_m', 'padiabe', 'pasight', 'papainlv', 'parjudg', 'padrinkbd', 'pacncrothr', 'palunglmt_m', 'parorgnz', 'pabreast', 'pastroke', 'pahrtatlmt', 'parxstrok', 'pahearaid', 'pashophlp', 'parxarthr', 'pawalkhlp', 'pabedhlp', 'paclims', 'paurina2y', 'pawheeze', 'pachair', 'padrinkb', 'paeathlp', 'paarms', 'pawthh', 'pastroklmt', 'pawalkr', 'pauppermoba', 'paagey', 'pabmi', 'pastoop', 'palowermoba', 'padresshlp', 'pamealhlp'